In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. EDA

In [ ]:
df = pd.read_csv('/kaggle/input/loan-default-prediction/Default_Fin.csv')
df.head()

In [ ]:
df.info()

In [ ]:
## Predicted variable distribution
df['Defaulted?'].value_counts().plot(kind='bar')
plt.title('Figure 1: Not default vs Default')
plt.show()

In [ ]:
#Predicted variable distribution per employment type
sns.barplot(x='Employed', y='Defaulted?', data=df)
plt.title('Default distribution by Employment type')
plt.show()

In [ ]:
##Bank Balance distribution per default status:
plt.figure(figsize=(12,6))
plt.hist(df[df['Defaulted?']==1]['Bank Balance'], histtype='step', bins=100, label='Defaulted')
plt.hist(df[df['Defaulted?']==0]['Bank Balance'], histtype='step', bins=100, label='Not Defaulted')
plt.title('Figure 3: Bank Balance distribution by Default status')
plt.legend()
plt.show()

In [ ]:
##Annual Salary distribution per default status:
plt.figure(figsize=(12,6))
plt.hist(df[df['Defaulted?']==1]['Annual Salary'], histtype='step', bins=100, label='Defaulted')
plt.hist(df[df['Defaulted?']==0]['Annual Salary'], histtype='step', bins=100, label='Not Defaulted')
plt.title('Figure 4: Annual Salary distribution by Default status')
plt.legend()
plt.show()

In [ ]:
##Annual Salary versus Bank balance per default type
plt.figure(figsize=(18,8))
sns.scatterplot(x='Annual Salary', y='Bank Balance', hue='Defaulted?', data=df, size='Employed', style='Employed')
plt.title('Figure 5: Annual Salary versus Bank balance per default types')
plt.show()


EDA Conclusions: 
1. From Figure 5, we can see that most defaulted cases appears in the upper part of the scatter plots. The upper part represent customers whose bank balance is relatively large compared to their annual salary, meaning they may have taken on larger debt in proportion with their salary than they should. This is consistent with a reality-based observation: if one take on debt more than they can afford, they are more likely to default.
2. In Figure 5, the bigger markers - representing unemployed customer - gather in the left side of the chart. This indicates that most unemployed customers have a lower range of annual salary. Despite the lower salary range, their bank balance distribution is similar to the employed group. Therefore we can deduce that, compared to the employed group, there is a higher proportion of unemployed customers borrowing more than they should. Therefore, statistically unemployed customer is more likely to default. This hypothesis is also supported by Figure 2, where we can see the Unemployed customer group has a higher default mean of IQR.

Overall we can see that a tree-based algorithm or support vector machine will do well in this problem.

# 2. Model Selection

In [ ]:
##Split data:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['Index', 'Defaulted?'])
y = df['Defaulted?']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10, stratify=y)

As tree-based model is non-parameter, it's not neccessary to do any scaling. The varibles are also coded in the right (numerical) format, so I'll go ahead and perform cross validation with hyper parameter tuning. 

But before doing hyperparameter tuning, I'm curious if random forest will be better than gradient boosted tree. I also want to use a cost-sensitve SVM model as a baseline against these 2 tree based model, so I'll compare both of them by plotting the learning curve to see which one is more likely to overfit.

**Plot learning curve of Gradient Boosting Classifier and Random Forest Classifier, against SVM**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from sklearn.model_selection import learning_curve

In [ ]:
def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None, scoring='recall',
                        n_jobs=None, train_sizes=[0.2, 0.4, 0.6, 0.8]):
    """
    Generate the test and training learning curve

    Parameters
    ----------
    estimator : estimator instance

    title : strTitle for the chart.

    X :Training vector

    y : Target relative to ``X`` for classification or regression;

    axes : Axes object that is used for plotting the curves.

    ylim : tuple of shape (2,), default=None
        Defines minimum and maximum y-values plotted, e.g. (ymin, ymax).

    cv : int, cross-validation generator or an iterable, default=None
        

    n_jobs : int or None, default=None
        Number of jobs to run in parallel.

    train_sizes : array-like of shape (n_ticks,)
        Relative or absolute numbers of training examples that will be used to generate the learning curve. If the ``dtype`` is float, it is regarded
        as a fraction of the maximum size of the training set (that is determined by the selected validation method), i.e. it has to be within
        (0, 1]. Otherwise it is interpreted as absolute sizes of the trainingsets. Note that for classification the number of samples usually have
        to be big enough to contain at least one sample from each class.(default: np.linspace(0.1, 1.0, 5))
    """
    axes.set_title(title)
    if ylim is not None:
        axes.set_ylim(*ylim)
    axes.set_xlabel("Training examples")
    axes.set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, scoring=scoring, n_jobs=n_jobs,
                       train_sizes=train_sizes, return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    # Plot learning curve
    axes.grid()
    axes.fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes.fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes.plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes.plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes.legend(loc="best")

    return plt

In [ ]:
#Define 3 types of models:
gbc = GradientBoostingClassifier()
rfc = RandomForestClassifier()
svc = make_pipeline(StandardScaler(), SVC(class_weight={0:1, 1:100})) ##The class_weight parameters is a must as we are telling 
##the model to focus on training class 1 by giving it higher weight.

**Compare learning curve with recall as the metrics**

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 8))

#GBC
title = "Learning Curves (Gradient Boosting Classifier)"
estimator = gbc
plot_learning_curve(estimator, title, X_train, y_train, axes=axes[0], ylim=(0.2, 1.01),
                    cv=5)

#RFC
title = r"Learning Curves (Random Forest Classifier)"
estimator = rfc
plot_learning_curve(estimator, title, X_train, y_train, axes=axes[1], ylim=(0.2, 1.01),
                    cv=5)

#SVC
title = r"Learning Curves (Support Vector Classifier)"
estimator = svc
plot_learning_curve(estimator, title, X_train, y_train, axes=axes[2], ylim=(0, 1.01),
                    cv=5)

plt.show()

**Compare learning curve using accuracy as the metrics**

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 8))

title = "Learning Curves (Gradient Boosting Classifier)"
estimator = gbc
plot_learning_curve(estimator, title, X_train, y_train, axes=axes[0], ylim=(0.7, 1.01),
                    cv=5, scoring='accuracy')

title = r"Learning Curves (Random Forest Classifier)"
estimator = rfc
plot_learning_curve(estimator, title, X_train, y_train, axes=axes[1], ylim=(0.7, 1.01),
                    cv=5, scoring='accuracy' )

title = r"Learning Curves (Support Vector Classifier)"
estimator = svc
plot_learning_curve(estimator, title, X_train, y_train, axes=axes[2], ylim=(0.6, 1.01),
                    cv=5, scoring='accuracy' )

plt.show()

With larger training examples, the gap between training score and cross-validation score of GBC becomes smaller, while RFC's stay almost the same. As our training size is quite large (7000), I'll prefer the Gradient Boosting Classifer. 

Noted that the Support Vector Classifier does not overfit, which is quite normal as this model only relies on some vector machine.

Considering a default prediction excercise, recall is a more important metrics (busines-wide). I'll definitely go for a cost-sensitive support vector classifier. 


# 3. Hyper-parameter tuning

**Hyper-parameter tuning for Gradient Boosting Classifer**

Support Vector Classifier should be the optimal solution if 'recall' is the main metrics. However, since I'm so invested in the Trees, I'll tune the GBC to see if its recalls can improve. 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
#Define parameteres grid
#There are only 3 variables (employed, bank balance and annual salary), hence max_features is not that meaningful to tune. 
#For max_depth, I still want to tune it as it's a relatively important hyper-parameters for ensemble tree model. 
params = {
   'learning_rate' : [0.1, 0.3, 0.5, 0.7],
    'n_estimators': [100, 200, 300, 400, 500],
    'subsample': [0.4, 0.6, 0.8],
    'max_depth': [2, 3],
    'min_samples_leaf' :[10, 50, 100],
}

In [ ]:
#Perform Randomize search on training set, using 2 scoring metrics
clf = RandomizedSearchCV(gbc, param_distributions=params, n_iter=20, scoring =['accuracy', 'recall'], cv=5, 
                         refit='recall', random_state=10, return_train_score=True)
results = clf.fit(X_train, y_train)

In [ ]:
#Plot Random Search Results
def plot_random_search_results(results):
    '''
    Input:
    
    results: A randomized search cv object, after fitting with data
    
    Return: Chart visualising the train and test score of the Randomized search process.
    '''
    plt.figure(figsize=(12,5))
    plt.plot(results.cv_results_['mean_train_accuracy'], label = 'Mean Train Accuracy')
    plt.plot(results.cv_results_['mean_test_accuracy'], label = 'Cross-Val Accuracy')
    plt.plot(results.cv_results_['mean_train_recall'], label = 'Mean Train Recall')
    plt.plot(results.cv_results_['mean_test_recall'], label = 'Cross-Val Recall')


    plt.annotate('Model with best recall', xy=(results.best_index_, 0.58), xytext=(results.best_index_ + 0.05, 0.7),
            arrowprops=dict(facecolor='blue', shrink=0.15, width=2))

    plt.title('Train and Test Score in each interations')
    plt.xlabel('Score')
    plt.ylabel('RandomSearch Iterations')
    plt.legend(loc='best')
    return plt


In [ ]:
plot_random_search_results(results)
plt.show()

In [ ]:
results.best_estimator_

#I'm supposed to tune the parameter for SVC, however it took a lot of time to tune while current result I got is already above 95%, so I decided to skip this step.

# Evaluate

**Evaluate with Gradient Boosting Trees**

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, plot_roc_curve

y_pred = results.best_estimator_.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy score of best model on the test data is: %f' %accuracy)
recall = recall_score(y_test, y_pred)
print('Recall score of best model on the test data is: %f' %recall)

#Print AUC Curve
fig, ax = plt.subplots(1, 1, figsize=(10,6))
plot_roc_curve(results.best_estimator_, X_test, y_test, ax = ax) 
plt.title('ROC of best GBC model')
plt.show()

***Evaluate with Support Vector Machine**

In [ ]:
svc.fit(X_train, y_train)

In [ ]:
y_pred_svc = svc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_svc)
print('Accuracy score of best model on the test data is: %f' %accuracy)
recall = recall_score(y_test, y_pred_svc)
print('Recall score of best model on the test data is: %f' %recall)

#Print AUC Curve
fig, ax = plt.subplots(1, 1, figsize=(10,6))
plot_roc_curve(svc, X_test, y_test, ax = ax) 
plt.title('ROC of SVC model')
plt.show()

# Conclusion

We can see that while tree-based model perform better on average if accuracy is the main metrics; SVC is the best choice in this case where recall is used to score. 
This makes me wonder, if we combine both of this model, we will get a best of both world results. 